![](https://storage.googleapis.com/mle-courses-prod/users/61b6fa1ba83a7e37c8309756/private-files/26a44ab0-c425-11ef-99fc-e9ac20f73f15-LLMs_truyen_kieu.png)

Tác giả: [ProtonX Team](https://protonx.io/courses/66b04dce02b797001269359b)

In [59]:
from google.colab import userdata
from huggingface_hub import login

# Log in with your token
login(userdata.get("hugging_face_token"))

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [3]:
!wget --no-check-certificate \
    https://gist.githubusercontent.com/khacanh/4c4662fa226db87a4664dfc2f70bc63e/raw/5d8a1d890c73a1e92e6898137db28f3dc0676975/kieu.txt \
    -O ./kieu.txt

--2025-05-07 14:58:17--  https://gist.githubusercontent.com/khacanh/4c4662fa226db87a4664dfc2f70bc63e/raw/5d8a1d890c73a1e92e6898137db28f3dc0676975/kieu.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 135832 (133K) [text/plain]
Saving to: ‘./kieu.txt’

./kieu.txt          100%[===================>] 132.65K  --.-KB/s    in 0.003s  

2025-05-07 14:58:18 (41.2 MB/s) - ‘./kieu.txt’ saved [135832/135832]



![](https://storage.googleapis.com/mle-courses-prod/users/61b6fa1ba83a7e37c8309756/private-files/860e1cb0-c434-11ef-a3aa-4b5f419a5062-Screen_Shot_2024_12_27_at_16.25.08.png)

In [4]:
corpus = []
f = open("kieu.txt", "r")
for line in f:
  corpus.append(line)

In [5]:
corpus

['Trăm năm trong cõi người ta,\n',
 'Chữ tài chữ mệnh khéo là ghét nhau.\n',
 'Trải qua một cuộc bể dâu,\n',
 'Những điều trông thấy mà đau đớn lòng.\n',
 'Lạ gì bỉ sắc tư phong,\n',
 'Trời xanh quen thói má hồng đánh ghen.\n',
 'Cảo thơm lần giỞ trước đèn,\n',
 'Phong tình có lục còn truyền sử xanh.\n',
 'Rằng năm Gia Tĩnh triều Minh,\n',
 'Bốn phương phẳng lặng, hai kinh vững vàng.\n',
 'Có nhà viên ngoại họ Vương,\n',
 'Gia tư nghĩ cũng thường thường bực trung.\n',
 'Một trai con thứ rốt lòng,\n',
 'Vương Quan là chữ, nối dòng nho gia.\n',
 'Đầu lòng hai ả tố nga,\n',
 'Thúy Kiều là chị, em là Thúy Vân.\n',
 'Mai cốt cách, tuyết tinh thần,\n',
 'Một người một vẻ, mười phân vẹn mười.\n',
 'Vân xem trang trọng khác vời,\n',
 'Khuôn trăng đầy đặn, nét ngài nỞ nang.\n',
 'Hoa cười ngọc thốt đoan trang,\n',
 'Mây thua nước tóc, tuyết nhường màu da.\n',
 'Kiều càng sắc sảo, mặn mà,\n',
 'So bề tài, sắc, lại là phần hơn.\n',
 'Làn thu thủy, nét xuân sơn,\n',
 'Hoa ghen thua thắm, liễu hờn 

In [6]:
model_name = "gpt2"

In [7]:
import random

In [8]:
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import torch
import random

In [11]:
data = corpus
# Shuffle data and split into train/validation
random.seed(42)
random.shuffle(data)
split_idx = int(0.8 * len(data))
train_data = data[:split_idx]
val_data = data[split_idx:]

# Create Hugging Face Dataset
dataset = DatasetDict({
    "train": Dataset.from_dict({"text": train_data}),
    "validation": Dataset.from_dict({"text": val_data})
})

In [18]:
dataset["train"]["text"][:5]

['Ai ra đường nấy ai nhìn được ai.\n',
 'Tựa nương bên triện một mình thiu thiu.\n',
 'Chưa xong cuộc rượu lại bày trò chơi.\n',
 'Sinh rằng: “Phác họa vừa rồi,\n',
 'Tiểu thư đâu đã rẽ hoa bước vào.\n']

In [19]:
dataset["train"][0]

{'text': 'Ai ra đường nấy ai nhìn được ai.\n'}

In [20]:
dataset["train"][1]

{'text': 'Tựa nương bên triện một mình thiu thiu.\n'}

### Khám phá một mẫu

In [21]:
dataset["train"][0]

{'text': 'Ai ra đường nấy ai nhìn được ai.\n'}

In [22]:
# Tokenization
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # GPT2 doesn't have pad token


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Từ điển: Vocab

In [23]:
# Access and print all tokens
vocab = tokenizer.get_vocab()

# Print each token in the vocabulary
for token, index in vocab.items():
    print(f"{index}: {token}")

35763: ĠHirosh
35039: EEEE
16629: ĠMilwaukee
26968: bench
22694: Ġcatastrophic
36271: ĠVelvet
42070: Ġstocked
64: a
12981: ĠGill
46627: ĠCOURT
11066: ĠMas
13030: Ġadvised
10330: Ġmargin
23455: ĠDiscovery
7869: Ġstatistics
30358: ĠClaw
22753: Ġimpaired
36703: ĠMarketplace
17729: Ġ1080
38089: Ġ560
9924: izabeth
23694: ĠLuna
49468: meier
39143: ĠVariety
29344: Ġirrig
4555: Ġeth
44032: PIN
34824: Ġpatented
28199: ĠBradford
8978: Ġreaching
13611: Ġslower
31371: entin
19857: Forge
44579: toe
46554: Ġapopt
22106: Ġemergence
36113: Bush
21152: Ġapparatus
943: ĠAr
2764: ĠCount
25874: Ġglyph
5288: Ġminimum
17724: Ġprediction
22544: 005
26001: FB
34605: Forest
7220: user
49457: allo
29642: daughter
29307: votes
3574: ĠFrom
43120: uca
44372: uron
26785: ĠKag
47028: ĠYen
35837: Ġdancers
4968: Ġreject
18502: ĠChoice
13220: raham
23769: ĠCock
8298: 001
28316: Charg
33939: Democratic
32131: ĠPubMed
27836: ĠSpear
8026: ĠStone
18645: Ġboundary
17126: Stat
17851: Ġcategor
45222: Ġclassmate
16008: Ġcollea

In [28]:
token = tokenizer(dataset["train"][0]["text"])
token

{'input_ids': [32, 72, 2179, 34754, 239, 130, 108, 157, 119, 251, 782, 299, 157, 118, 98, 88, 257, 72, 299, 71, 127, 105, 77, 34754, 239, 130, 108, 157, 119, 96, 66, 257, 72, 13, 198], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [30]:
len(token["input_ids"])

35

In [26]:
dataset["train"][0]["text"]

'Ai ra đường nấy ai nhìn được ai.\n'

In [31]:
# Get the token IDs
input_ids = tokenizer(dataset["train"][0]["text"])["input_ids"]

# Decode the full sequence at once
decoded_text = tokenizer.decode(input_ids, skip_special_tokens=True)
print(decoded_text)
# Print each token separately
tokens = list(decoded_text)
print(tokens)

Ai ra đường nấy ai nhìn được ai.

['A', 'i', ' ', 'r', 'a', ' ', 'đ', 'ư', 'ờ', 'n', 'g', ' ', 'n', 'ấ', 'y', ' ', 'a', 'i', ' ', 'n', 'h', 'ì', 'n', ' ', 'đ', 'ư', 'ợ', 'c', ' ', 'a', 'i', '.', '\n']


In [33]:

def tokenize_function(examples):
    tokens = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/2603 [00:00<?, ? examples/s]

Map:   0%|          | 0/651 [00:00<?, ? examples/s]

In [34]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2603
    })
    validation: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 651
    })
})

In [45]:
def decode(input_ids):
  return tokenizer.decode(input_ids, skip_special_tokens=True)

In [43]:
tokenized_datasets["train"][0]["text"]

'Ai ra đường nấy ai nhìn được ai.\n'

In [44]:
decode(tokenized_datasets["train"][0]["input_ids"])

'Ai ra đường nấy ai nhìn được ai.\n<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|e

In [38]:
tokenized_datasets["train"][0]["input_ids"]

[32,
 72,
 2179,
 34754,
 239,
 130,
 108,
 157,
 119,
 251,
 782,
 299,
 157,
 118,
 98,
 88,
 257,
 72,
 299,
 71,
 127,
 105,
 77,
 34754,
 239,
 130,
 108,
 157,
 119,
 96,
 66,
 257,
 72,
 13,
 198,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256]

In [46]:
tokenized_datasets["train"]

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2603
})

In [47]:
tokenized_datasets["validation"]

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 651
})

In [48]:
# Load GPT Model
model = AutoModelForCausalLM.from_pretrained(model_name)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [60]:
# Training Arguments
training_args = TrainingArguments(
    output_dir="./gpt-pretrained",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=20,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    save_total_limit=2
)

from transformers import DataCollatorForLanguageModeling

# Data collator for causal language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Disable Masked Language Modeling (we are doing Causal LM)
)

# Trainer with updated collator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)


<ipython-input-60-19bacebab281>:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [61]:
# Train
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.386400,1.738363


TrainOutput(global_step=651, training_loss=1.2575977421759095, metrics={'train_runtime': 187.4094, 'train_samples_per_second': 13.889, 'train_steps_per_second': 3.474, 'total_flos': 170035789824000.0, 'train_loss': 1.2575977421759095, 'epoch': 1.0})

In [62]:
# Generate text
model.eval()
input_text = "Trăm năm"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_length=100,
        num_return_sequences=1,
        temperature=0.7,
        top_p=0.95,
        pad_token_id=tokenizer.eos_token_id
    )

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated text:")
print(generated_text)

# "Trăm năm" -> "trong"
# "Trăm năm trong" -> "cõi"
# "Trăm năm trong cõi" -> "người"

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Generated text:
Trăm năm để một lạ đời,

Ở đây xưa.

Ở đây xưa.

�010 đây xưa.Theo,

Ở đAlong.

�010 đời xưa.



In [63]:
trainer.push_to_hub(commit_message = "Training completed!")

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Dat1710/gpt-pretrained/commit/68b0f9b86153c670ee098120bbde67376fabf8ea', commit_message='Training completed!', commit_description='', oid='68b0f9b86153c670ee098120bbde67376fabf8ea', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Dat1710/gpt-pretrained', endpoint='https://huggingface.co', repo_type='model', repo_id='Dat1710/gpt-pretrained'), pr_revision=None, pr_num=None)

In [64]:
from transformers import pipeline
# Change `Dat1710` to your Hub username
model_id = "Dat1710/gpt-pretrained"
classifier = pipeline("text-generation", model=model_id)

config.json:   0%|          | 0.00/880 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

Device set to use cuda:0


In [72]:
custom_tweet = "Trăm năm"
preds = classifier(custom_tweet)
print(preds[0]["generated_text"])

Trăm năm năm đôi lạc một vì đồng!

ứng chiêm luố


In [68]:
preds[0]["generated_text"]

'Trăm năm, năm thấy bỉ đường,\n\nề.\n\niều đư�'